In [1]:
import pandas as pd
import re
import requests

In [2]:
df = pd.read_excel('南方日报_微博用户.xlsx')

In [3]:
df['full_text_url']

0                              /comment/P6H5PikLZ?ckAll=1
1                              /comment/P6GVqANQJ?ckAll=1
2       https://vote.weibo.com/h5/index/index?vote_id=...
3                              /comment/P6GL1v6ec?ckAll=1
4       https://vote.weibo.com/h5/index/index?vote_id=...
                              ...                        
4886                           /comment/OxAfAznoQ?ckAll=1
4887                           /comment/OxA59eAJm?ckAll=1
4888                           /comment/OxzLl5cZS?ckAll=1
4889                           /comment/Oxzknt8bY?ckAll=1
4890                           /comment/OxyZNjOH9?ckAll=1
Name: full_text_url, Length: 4891, dtype: object

In [4]:
# 处理函数
def process_url(x):
    if isinstance(x, str):  # 先检查是否是字符串类型
        x = x.strip()  # 去除两端空格
        if x.startswith('/comment/'):  # 判断是否以 /comment/ 开头
            return 'https://weibo.cn' + x
        else:
            return None
    else:
        return None  # 非字符串类型（比如 None 或其他）

In [ ]:
# 应用处理函数
df['full_text_url_true'] = df['full_text_url'].apply(process_url)

In [6]:
df['full_text_url_true']

0       https://weibo.cn/comment/P6H5PikLZ?ckAll=1
1       https://weibo.cn/comment/P6GVqANQJ?ckAll=1
2                                             None
3       https://weibo.cn/comment/P6GL1v6ec?ckAll=1
4                                             None
                           ...                    
4886    https://weibo.cn/comment/OxAfAznoQ?ckAll=1
4887    https://weibo.cn/comment/OxA59eAJm?ckAll=1
4888    https://weibo.cn/comment/OxzLl5cZS?ckAll=1
4889    https://weibo.cn/comment/Oxzknt8bY?ckAll=1
4890    https://weibo.cn/comment/OxyZNjOH9?ckAll=1
Name: full_text_url_true, Length: 4891, dtype: object

In [7]:
# 去掉 NaN 值并将结果转为列表
url_list = df['full_text_url_true'].dropna().tolist()

# 打印结果
print(url_list)


['https://weibo.cn/comment/P6H5PikLZ?ckAll=1', 'https://weibo.cn/comment/P6GVqANQJ?ckAll=1', 'https://weibo.cn/comment/P6GL1v6ec?ckAll=1', 'https://weibo.cn/comment/P6GBd8nDg?ckAll=1', 'https://weibo.cn/comment/P6GwKwgkw?ckAll=1', 'https://weibo.cn/comment/P6GsSxEGu?ckAll=1', 'https://weibo.cn/comment/P6Gjn4ALW?ckAll=1', 'https://weibo.cn/comment/P6Gb91eRa?ckAll=1', 'https://weibo.cn/comment/P6G9z7c77?ckAll=1', 'https://weibo.cn/comment/P6G2feznm?ckAll=1', 'https://weibo.cn/comment/P6EO08NgR?ckAll=1', 'https://weibo.cn/comment/P6FN1y65J?ckAll=1', 'https://weibo.cn/comment/P6FAnAycX?ckAll=1', 'https://weibo.cn/comment/P6FwPdOJc?ckAll=1', 'https://weibo.cn/comment/P6FsREnCD?ckAll=1', 'https://weibo.cn/comment/P6FeQi5ig?ckAll=1', 'https://weibo.cn/comment/P6Fd3sjlq?ckAll=1', 'https://weibo.cn/comment/P6F6phVQR?ckAll=1', 'https://weibo.cn/comment/P6F12wap3?ckAll=1', 'https://weibo.cn/comment/P6EO08NgR?ckAll=1', 'https://weibo.cn/comment/P6EgBqNYT?ckAll=1', 'https://weibo.cn/comment/P6Edu5c

In [8]:
len(url_list)

3417

# 函数：请求数据

In [25]:
import requests

def get_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
        "cookie": 'SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWreg3M85waDCfwhLTHsUGS5JpX5KMhUgL.FoMX1h-p1hzfe0q2dJLoIp7LxKML1KBLBKnLxKqL1hnLBoMceh5EShe4S0nf; ALF=1737460694; SUB=_2A25KY46GDeRhGeFK41cQ-CzJyDqIHXVpAI5OrDV8PUJbkNAbLRPNkW1NQsp0yEnMqGgG3mpiBu9fpzLIyGAoKnTl'
    }

    try:
        # 发起请求
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # 如果请求失败（非200响应），会抛出异常

        # 获取 HTML 内容，并去掉前面的声明
        html = response.text[38:]
        
        return html
    except requests.exceptions.RequestException as e:
        # 如果请求发生异常，打印错误信息并返回 None
        print(f"请求发生错误: {e}")
        return None


# 函数：解析数据

In [10]:
from pyquery import PyQuery as pq

def extract_weibo_fulltext(html):
    doc = pq(html)
    fulltext_dict = {'fulltext': None}

    try:
        fulltext_dict['fulltext'] = doc('span.ctt').text()

    except Exception as e:
        print(f"解析数据失败: {e}")

    return fulltext_dict

# 测试

In [21]:
url = url_list[0]  # 取第一个 url 进行示例
url

'https://weibo.cn/comment/P6H5PikLZ?ckAll=1'

In [22]:
html = get_html(url)
html

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta http-equiv="Cache-Control" content="no-cache"/><meta id="viewport" name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0, maximum-scale=2.0" /><link rel="shortcut icon" type="image/x-icon" href="https://weibo.cn/favicon.ico"><link rel="icon" sizes="any" mask href="https://h5.sinaimg.cn/upload/2015/05/15/28/WeiboLogoCh.svg" color="black"><meta name="MobileOptimized" content="240"/><title>评论列表</title><style type="text/css" id="internalStyle">html,body,p,form,div,table,textarea,input,span,select{font-size:12px;word-wrap:break-word;}body{background:#F8F9F9;color:#000;padding:1px;margin:1px;}table,tr,td{border-width:0px;margin:0px;padding:0px;}form{margin:0px;padding:0px;border:0px;}textarea{border:1px solid #96c1e6}textarea{width:95%;

In [23]:
extract_weibo_fulltext(html)

{'fulltext': ':【#专家提醒黑色染发剂最危险#】年过50的李女士近十年频繁染发，近段时间，李女士因身体疲乏、皮肤暗淡而到了医院就诊，结果被查出肝硬化中晚期。医生分析，李女士的肝硬化与长期频繁染发有关。\n\n来自美国相关机构的研究发现，使用永久染发剂与卵巢病、皮肤癌、淋巴瘤乳腺癌等癌症风险增加有关。黑色染发剂中的苯二胺类物质、重金属盐是最多的，因此是最危险的。（都市报道） 南方日报的微博视频 怎样算频繁呢？ 没有安全的染发剂吗？'}

In [24]:
import random
import time

all_weibo_list = [] # 存储所有微博的列表

for url in url_list[:5]:

    print(f"正在抓取：{url}")
    html = get_html(url) # 获取 HTML 内容

    fulltext_dict = extract_weibo_fulltext(html) # 提取微博全文内容
    fulltext_dict["full_text_url"] = url[16:] # 增加 URL 信息
    
    wait_time = random.randint(3, 5) # 生成 3 到 5 秒的随机浮点数
    print(f"等待{wait_time}秒")
    time.sleep(wait_time)   # 暂停指定的秒数
    print("-"*50)
        
    all_weibo_list.append(fulltext_dict) # 将当前页面的微博列表添加到总的列表中

print("抓取完成")


正在抓取：https://weibo.cn/comment/P6H5PikLZ?ckAll=1
等待4秒
--------------------------------------------------
正在抓取：https://weibo.cn/comment/P6GVqANQJ?ckAll=1
等待3秒
--------------------------------------------------
正在抓取：https://weibo.cn/comment/P6GL1v6ec?ckAll=1
等待4秒
--------------------------------------------------
正在抓取：https://weibo.cn/comment/P6GBd8nDg?ckAll=1
等待3秒
--------------------------------------------------
正在抓取：https://weibo.cn/comment/P6GwKwgkw?ckAll=1
等待4秒
--------------------------------------------------
抓取完成


In [15]:
all_weibo_list

[{'fulltext': ':【#专家提醒黑色染发剂最危险#】年过50的李女士近十年频繁染发，近段时间，李女士因身体疲乏、皮肤暗淡而到了医院就诊，结果被查出肝硬化中晚期。医生分析，李女士的肝硬化与长期频繁染发有关。\n\n来自美国相关机构的研究发现，使用永久染发剂与卵巢病、皮肤癌、淋巴瘤乳腺癌等癌症风险增加有关。黑色染发剂中的苯二胺类物质、重金属盐是最多的，因此是最危险的。（都市报道） 南方日报的微博视频 怎样算频繁呢？ 没有安全的染发剂吗？',
  'full_text_url': '/comment/P6H5PikLZ?ckAll=1'},
 {'fulltext': ':#大疆拟建250米超高层总部大楼#【大疆拟深圳湾超总拿地，建250米+超高层总部大楼！】12月25日，深圳市政府官网发布《智能航空系统产业生态全球总部项目遴选方案》显示，宗地位于深圳湾超级总部基地DU01-02地块，用地面积15658平方米，总建筑面积188000㎡。\n\n目前意向用地单位为深圳市大疆创新科技有限公司。根据公示，建成后，100%建筑面积在30年全出让年期内不得转让，办公部分仅可供大疆系及大疆系控股企业使用。\n\n参考今年6月，南山区发改委对该地块征求意见稿，该地块位于白石四道和深湾二路交汇处，招商银行全球总部大厦东侧，紧贴深超总公园绿轴，限高300米，欲建设250米超高层。',
  'full_text_url': '/comment/P6GVqANQJ?ckAll=1'},
 {'fulltext': ':【深大乒超俱乐部新赛季启航！#孙颖莎称相信能打好今年最后一战#】12月26日，“深圳大学乒超俱乐部代表队2024赛季启航仪式”在深圳市南山区蛇口文体中心举行，深圳大学乒超俱乐部队员孙颖莎、覃予萱、吴咏琳出席活动。\n\n#孙颖莎平常用聊天式沟通准备好比赛#对于12月30日将于上海举行的2024年中国乒超联赛总决赛，孙颖莎在接受采访时表示，俱乐部队员们平时会以“聊天式”的沟通把赛前的准备安排好，对于今年最后一场比赛，她相信自己和队友都会齐心协力完成好比赛。#孙颖莎[超话]# 南方日报的微博视频 加油 小莎加油深大加油！！！ “她”力量无穷，言之有物，言之有情，言之有理。我真爱听她发言，有格局又不失幽默。',
  'full_text_url': '/comment/P6

# 正式抓取

In [ ]:
import random
import time

all_weibo_list = [] # 存储所有微博的列表

for url in url_list:

    print(f"正在抓取：{url}")
    html = get_html(url) # 获取 HTML 内容

    fulltext_dict = extract_weibo_fulltext(html) # 提取微博全文内容
    fulltext_dict["full_text_url"] = url[16:] # 增加 URL 信息
    
    wait_time = random.randint(3, 5) # 生成 3 到 5 秒的随机浮点数
    print(f"等待{wait_time}秒")
    time.sleep(wait_time)   # 暂停指定的秒数
    
    print("-"*50)
    
    
    all_weibo_list.append(fulltext_dict) # 将当前页面的微博列表添加到总的列表中
    
    print(f"已抓取:{len(all_weibo_list)}")
    
print("抓取完成")


# 检查数据

In [27]:
all_weibo_list

[{'fulltext': ':【#专家提醒黑色染发剂最危险#】年过50的李女士近十年频繁染发，近段时间，李女士因身体疲乏、皮肤暗淡而到了医院就诊，结果被查出肝硬化中晚期。医生分析，李女士的肝硬化与长期频繁染发有关。\n\n来自美国相关机构的研究发现，使用永久染发剂与卵巢病、皮肤癌、淋巴瘤乳腺癌等癌症风险增加有关。黑色染发剂中的苯二胺类物质、重金属盐是最多的，因此是最危险的。（都市报道） 南方日报的微博视频 怎样算频繁呢？ 没有安全的染发剂吗？',
  'full_text_url': '/comment/P6H5PikLZ?ckAll=1'},
 {'fulltext': ':#大疆拟建250米超高层总部大楼#【大疆拟深圳湾超总拿地，建250米+超高层总部大楼！】12月25日，深圳市政府官网发布《智能航空系统产业生态全球总部项目遴选方案》显示，宗地位于深圳湾超级总部基地DU01-02地块，用地面积15658平方米，总建筑面积188000㎡。\n\n目前意向用地单位为深圳市大疆创新科技有限公司。根据公示，建成后，100%建筑面积在30年全出让年期内不得转让，办公部分仅可供大疆系及大疆系控股企业使用。\n\n参考今年6月，南山区发改委对该地块征求意见稿，该地块位于白石四道和深湾二路交汇处，招商银行全球总部大厦东侧，紧贴深超总公园绿轴，限高300米，欲建设250米超高层。',
  'full_text_url': '/comment/P6GVqANQJ?ckAll=1'},
 {'fulltext': ':【深大乒超俱乐部新赛季启航！#孙颖莎称相信能打好今年最后一战#】12月26日，“深圳大学乒超俱乐部代表队2024赛季启航仪式”在深圳市南山区蛇口文体中心举行，深圳大学乒超俱乐部队员孙颖莎、覃予萱、吴咏琳出席活动。\n\n#孙颖莎平常用聊天式沟通准备好比赛#对于12月30日将于上海举行的2024年中国乒超联赛总决赛，孙颖莎在接受采访时表示，俱乐部队员们平时会以“聊天式”的沟通把赛前的准备安排好，对于今年最后一场比赛，她相信自己和队友都会齐心协力完成好比赛。#孙颖莎[超话]# 南方日报的微博视频 加油 小莎加油深大加油！！！ “她”力量无穷，言之有物，言之有情，言之有理。我真爱听她发言，有格局又不失幽默。',
  'full_text_url': '/comment/P6

In [28]:
len(all_weibo_list)

3417

# 导出数据

In [29]:
import pandas as pd

df = pd.DataFrame(all_weibo_list)

df.to_excel('南方日报_微博用户_全文.xlsx', index=False)